In [102]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from tqdm import tqdm

from resnet import ResNet18

In [103]:
torch.manual_seed(42)
device = 'cpu'
print(f"Using device: {device}")

Using device: cpu


In [104]:
def get_data_loaders(batch_size=128, num_workers=2):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=num_workers)

    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    return trainloader, testloader, classes

In [105]:
def train(model, trainloader, criterion, optimizer, scheduler, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    loop = tqdm(trainloader, desc=f"Epoch {epoch + 1}")
    for batch_idx, (inputs, targets) in enumerate(loop):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        loop.set_postfix(loss=train_loss / (batch_idx + 1), acc=100. * correct / total)

    scheduler.step()

    return train_loss / len(trainloader), 100. * correct / total

In [106]:
def val(model, testloader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return test_loss / len(testloader), 100.0 * correct / total

In [107]:
model = ResNet18().to(device)
model.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [108]:
trainloader, testloader, classes = get_data_loaders()
criterion = nn.CrossEntropyLoss()
test_loss, test_acc = val(model, testloader, criterion)
# model.eval()

In [109]:
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

Test Loss: 0.2110, Test Acc: 94.12%


In [110]:
def denormalize(tensor):
    mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
    std = torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
    return tensor * std + mean